In [2]:
import pathlib

import cartopy.feature as cfeature
import cf_xarray.units
import napari
import numpy as np
import pint_xarray
import shapely
import xarray as xr

# ds = xr.open_dataset("https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/gpcp-feedstock/gpcp.zarr", engine="zarr", consolidated=True).load()
# pathlib.Path("data").mkdir(exist_ok=True)
# ds.drop_encoding().to_zarr("data/gpcp.zarr", mode="w")


In [74]:
ds = xr.open_dataset(pathlib.Path() / "data/gpcp.zarr", engine="zarr").load()

# convert to [-180, 180] longitude because coastlines are in this range
ds = ds.assign_coords(longitude=lambda ds: (ds.longitude + 180) % 360 - 180).sortby("longitude")
ds

<xarray.Dataset> Size: 2GB
Dimensions:      (time: 9226, latitude: 180, longitude: 360, nv: 2)
Coordinates:
  * latitude     (latitude) float32 720B -90.0 -89.0 -88.0 ... 87.0 88.0 89.0
    lon_bounds   (longitude, nv) float32 3kB 180.0 181.0 181.0 ... 179.0 180.0
    lat_bounds   (latitude, nv) float32 1kB -90.0 -89.0 -89.0 ... 89.0 89.0 90.0
  * time         (time) datetime64[ns] 74kB 1996-10-01 1996-10-02 ... 2021-12-31
    time_bounds  (time, nv) datetime64[ns] 148kB 1996-10-01 ... 2022-01-01
  * longitude    (longitude) float32 1kB -180.0 -179.0 -178.0 ... 178.0 179.0
Dimensions without coordinates: nv
Data variables:
    precip       (time, latitude, longitude) float32 2GB 0.4359 0.4285 ... 0.0
Attributes: (12/45)
    Conventions:                CF-1.6, ACDD 1.3
    Metadata_Conventions:       CF-1.6, Unidata Dataset Discovery v1.0, NOAA ...
    acknowledgment:             This project was supported in part by a grant...
    cdm_data_type:              Grid
    cdr_program:                NOAA Climate Data Record Program for satellit...
    cdr_variable:               precipitation
    ...                         ...
    standard_name_vocabulary:   CF Standard Name Table (v41, 22 February 2017)
    summary:                    Global Precipitation Climatology Project (GPC...
    time_coverage_duration:     P1D
    time_coverage_end:          1996-10-01T23:59:59Z
    time_coverage_start:        1996-10-01T00:00:00Z
    title:                      Global Precipitation Climatatology Project (G...

In [114]:
# Swap coords from [lat, lon] to [lon, lat]
coastline_shapes = [shapely.get_coordinates(geom)[:, [1, 0]] for geom in cfeature.COASTLINE.geometries()]

In [109]:

def get_scale_translate(dataset, array_name):
    """Get the translate/offset and scale parameters for an xarray dataset.

    This code assumes that the dataset is regularly spaced. You should
    interpolate your data if it is sampled at irregular spaces.

    Parameters
    ----------
    dataset : xr.Dataset
        The dataset containing the array to be displayed.
    array_name : str
        The name of the xarray DataArray within `dataset` to be displayed in
        napari.

    Returns
    -------
    param_dict : dict[str, list[float]]
        The scale and translate parameters computed from the xarray dimension
        indices.
    """
    array = getattr(dataset, array_name)
    if array is None:
        raise ValueError(f'{dataset} has no array with name {array_name}')
    dims = [getattr(dataset, dim) for dim in array.dims]
    translate = [float(d[0]) for d in dims]
    scale = [float(d[1] - d[0]) for d in dims]
    return {'scale': scale, 'translate': translate}

kwargs = get_scale_translate(ds, 'precip')
kwargs

{'scale': [86400000000000.0, 1.0, 1.0],
 'translate': [8.44128e+17, -90.0, -180.0]}

In [115]:
viewer = napari.Viewer()
viewer.camera.orientation2d = ('up', 'right')
viewer.reset_view(margin=0.05)

In [116]:
viewer.add_image(
    ds['precip'].data,
    colormap='viridis',
    name=ds['precip'].attrs['standard_name'],
    contrast_limits=ds['precip'].attrs['valid_range'],
    **get_scale_translate(ds, 'precip'),
)
viewer.dims.axis_labels = ds['precip'].dims

In [117]:
viewer.add_shapes(
    coastline_shapes,
    shape_type='path',
    edge_color='white',
    edge_width=1,
    name='Coastlines',
)
viewer.reset_view()